# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\16.jpg,1
1,../dataset/wrist_xray/Normal\72.jpg,1
2,../dataset/wrist_xray/Fracture\9 Male (A View)...,0
3,../dataset/wrist_xray/Fracture\54 Male (L View...,0
4,../dataset/wrist_xray/Fracture\37 Male (L View...,0


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

d:\tutorials\automl\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# SqueezeNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.680   0.654     0.662
Class 1       0.492   0.510     0.490
macro         0.584   0.582     0.576
weighted      0.612   0.608     0.604
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.632   0.650     0.634
Class 1       0.446   0.448     0.434
macro         0.536   0.546     0.532
weighted      0.566   0.558     0.552
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.626   0.728     0.662
Class 1       0.466   0.378     0.394
macro         0.544   0.552     0.524
weighted      0.572   0.568     0.552
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.652   0.720     0.680
Class 1       0.476   0.424     0.440
macro         0.564   0.570     0.562
weighted      0.596   0.612     0.598
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.87,0.79,0.79,0.85,0.85
1,0.64,0.59,0.56,0.49,0.69
2,0.51,0.44,0.49,0.59,0.59
3,0.55,0.47,0.50,0.58,0.68
4,0.47,0.50,0.50,0.55,0.61
mean,0.608,0.558,0.568,0.612,0.684


In [9]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.638   0.632     0.630
Class 1       0.452   0.454     0.444
macro         0.546   0.542     0.538
weighted      0.576   0.574     0.568
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.590   0.582     0.584
Class 1       0.394   0.400     0.394
macro         0.494   0.490     0.490
weighted      0.522   0.518     0.516
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.596   0.666     0.624
Class 1       0.422   0.348     0.376
macro         0.510   0.506     0.500
weighted      0.538   0.544     0.532
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.606   0.676     0.630
Class 1       0.478   0.402     0.420
macro         0.540   0.536     0.522
weighted      0.568   0.552     0.544
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.79,0.72,0.69,0.54,0.82
1,0.46,0.51,0.51,0.54,0.72
2,0.59,0.41,0.46,0.54,0.62
3,0.58,0.53,0.61,0.61,0.68
4,0.45,0.42,0.45,0.53,0.66
mean,0.574,0.518,0.544,0.552,0.7


# ShuffleNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.586   0.598     0.572
Class 1       0.394   0.406     0.374
macro         0.488   0.502     0.474
weighted      0.520   0.500     0.488
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.540   0.610     0.564
Class 1       0.356   0.302     0.318
macro         0.448   0.456     0.440
weighted      0.478   0.478     0.466
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.560   0.614     0.580
Class 1       0.394   0.346     0.358
macro         0.476   0.480     0.470
weighted      0.506   0.498     0.492
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.582   0.708     0.634
Class 1       0.446   0.318     0.354
macro         0.512   0.512     0.494
weighted      0.538   0.540     0.520
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.44,0.44,0.46,0.49,0.59
1,0.44,0.59,0.54,0.62,0.77
2,0.49,0.38,0.46,0.49,0.67
3,0.68,0.53,0.53,0.63,0.68
4,0.45,0.45,0.50,0.47,0.58
mean,0.5,0.478,0.498,0.54,0.658


In [11]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.562   0.544     0.548
Class 1       0.418   0.430     0.414
macro         0.492   0.490     0.482
weighted      0.518   0.494     0.498
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.602   0.668     0.622
Class 1       0.490   0.406     0.430
macro         0.544   0.536     0.528
weighted      0.570   0.554     0.546
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.592   0.694     0.636
Class 1       0.452   0.352     0.388
macro         0.522   0.522     0.514
weighted      0.552   0.550     0.538
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.580   0.648     0.604
Class 1       0.432   0.364     0.382
macro         0.506   0.508     0.492
weighted      0.536   0.518     0.514
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.44,0.59,0.64,0.54,0.67
1,0.46,0.67,0.62,0.54,0.72
2,0.49,0.54,0.46,0.51,0.62
3,0.50,0.50,0.53,0.55,0.63
4,0.58,0.47,0.50,0.45,0.61
mean,0.494,0.554,0.55,0.518,0.65


# Mnasnet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.636   0.680     0.650
Class 1       0.536   0.480     0.496
macro         0.584   0.580     0.576
weighted      0.610   0.598     0.596
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.616   0.652     0.628
Class 1       0.488   0.458     0.466
macro         0.550   0.556     0.546
weighted      0.576   0.566     0.564
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.614   0.652     0.626
Class 1       0.480   0.458     0.458
macro         0.546   0.554     0.544
weighted      0.570   0.560     0.558
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.614   0.752     0.660
Class 1       0.472   0.342     0.364
macro         0.544   0.548     0.512
weighted      0.566   0.566     0.530
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.49,0.49,0.44,0.46,0.56
1,0.67,0.67,0.69,0.67,0.74
2,0.54,0.54,0.54,0.49,0.69
3,0.63,0.66,0.66,0.66,0.76
4,0.66,0.47,0.47,0.55,0.71
mean,0.598,0.566,0.56,0.566,0.692


In [13]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.562   0.580     0.562
Class 1       0.420   0.392     0.392
macro         0.490   0.488     0.478
weighted      0.520   0.492     0.494
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.572   0.720     0.630
Class 1       0.416   0.270     0.314
macro         0.492   0.494     0.474
weighted      0.518   0.532     0.502
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.568   0.618     0.582
Class 1       0.424   0.366     0.376
macro         0.496   0.492     0.480
weighted      0.524   0.506     0.502
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.580   0.614     0.584
Class 1       0.434   0.394     0.398
macro         0.506   0.504     0.488
weighted      0.534   0.512     0.508
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.56,0.49,0.51,0.51,0.56
1,0.41,0.62,0.51,0.44,0.62
2,0.49,0.44,0.38,0.51,0.62
3,0.53,0.61,0.58,0.63,0.66
4,0.47,0.50,0.55,0.47,0.61
mean,0.492,0.532,0.506,0.512,0.614


# MobileNet

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.646   0.600     0.610
Class 1       0.478   0.534     0.488
macro         0.560   0.566     0.548
weighted      0.590   0.570     0.568
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.592   0.670     0.616
Class 1       0.464   0.380     0.390
macro         0.528   0.526     0.506
weighted      0.560   0.540     0.526
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.632   0.634     0.620
Class 1       0.488   0.494     0.466
macro         0.560   0.566     0.544
weighted      0.590   0.560     0.556
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.546   0.558     0.536
Class 1       0.392   0.372     0.364
macro         0.468   0.464     0.448
weighted      0.498   0.466     0.464
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.67,0.64,0.67,0.46,0.74
1,0.56,0.49,0.51,0.33,0.72
2,0.51,0.46,0.46,0.49,0.64
3,0.53,0.58,0.63,0.58,0.66
4,0.58,0.53,0.53,0.47,0.63
mean,0.57,0.54,0.56,0.466,0.678


In [15]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.586   0.670     0.614
Class 1       0.430   0.366     0.372
macro         0.510   0.516     0.494
weighted      0.536   0.528     0.516
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.590   0.702     0.626
Class 1       0.422   0.334     0.348
macro         0.504   0.516     0.486
weighted      0.534   0.530     0.512
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.584   0.662     0.612
Class 1       0.432   0.362     0.380
macro         0.506   0.510     0.494
weighted      0.534   0.524     0.518
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.596   0.630     0.602
Class 1       0.472   0.426     0.422
macro         0.530   0.528     0.512
weighted      0.562   0.534     0.530
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.46,0.56,0.41,0.54,0.64
1,0.62,0.51,0.59,0.51,0.72
2,0.51,0.44,0.46,0.62,0.56
3,0.55,0.53,0.53,0.50,0.66
4,0.50,0.61,0.63,0.50,0.68
mean,0.528,0.53,0.524,0.534,0.652


# DenseNet121

In [16]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.638   0.666     0.646
Class 1       0.492   0.470     0.474
macro         0.564   0.568     0.560
weighted      0.594   0.600     0.592
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.702   0.730     0.712
Class 1       0.592   0.548     0.564
macro         0.648   0.642     0.638
weighted      0.674   0.668     0.668
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.740   0.738     0.736
Class 1       0.636   0.632     0.628
macro         0.690   0.686     0.684
weighted      0.710   0.702     0.698
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.638   0.652     0.640
Class 1       0.514   0.498     0.498
macro         0.576   0.576     0.568
weighted      0.604   0.590     0.590
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.59,0.67,0.72,0.46,0.69
1,0.64,0.56,0.64,0.56,0.64
2,0.51,0.56,0.62,0.54,0.62
3,0.58,0.68,0.74,0.68,0.74
4,0.68,0.87,0.79,0.71,0.87
mean,0.6,0.668,0.702,0.59,0.712


In [17]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.596   0.586     0.588
Class 1       0.428   0.438     0.428
macro         0.510   0.512     0.506
weighted      0.538   0.524     0.526
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.642   0.688     0.660
Class 1       0.526   0.470     0.492
macro         0.584   0.576     0.576
weighted      0.612   0.598     0.600
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.696   0.740     0.712
Class 1       0.628   0.570     0.588
macro         0.660   0.654     0.652
weighted      0.684   0.670     0.670
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.662   0.728     0.686
Class 1       0.578   0.496     0.522
macro         0.622   0.612     0.604
weighted      0.646   0.622     0.620
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.44,0.59,0.59,0.59,0.69
1,0.56,0.46,0.67,0.49,0.69
2,0.49,0.59,0.64,0.64,0.69
3,0.47,0.61,0.66,0.63,0.66
4,0.66,0.74,0.79,0.76,0.76
mean,0.524,0.598,0.67,0.622,0.698
